In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [68.5 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.download.nvi

     |████████████████████████████████| 788 kB 8.4 MB/s 
     |████████████████████████████████| 370 kB 47.8 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: IPython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from manim import *

import random

Manim Community v0.10.0

In [ ]:
x1 = np.linspace(1, 9, 9).tolist()
x2 = np.linspace(0, 0.9, 9).tolist()

# round the input so the data looks pretty
x1 = np.array(list(map(lambda x: round(x, 1), x1)))
np.random.shuffle(x1)
x2 = np.array(list(map(lambda x: round(x, 1), x2)))
np.random.shuffle(x2)
x = np.stack((x1,x2), 1)

x_shuffled = x[:]
np.random.shuffle(x_shuffled)
print(x_shuffled)

[[6.  0.9]
 [8.  0. ]
 [9.  0.3]
 [4.  0.5]
 [1.  0.1]
 [7.  0.6]
 [2.  0.2]
 [5.  0.7]
 [3.  0.8]]


In [ ]:
# An Mobject for drawing Neural Networks
# Contains modifications for Manim Community Edition
# Original source: https://www.youtube.com/watch?v=HnIeAP--vWc

import itertools as it
from numpy.lib.function_base import iterable

# A customizable Sequential Neural Network


class NeuralNetworkMobject(VGroup):
    # Remove CONFIG since it is now deprecated in ManimCE
    # CONFIG = {
    #     "neuron_radius": 0.15,
    #     "neuron_to_neuron_buff": MED_SMALL_BUFF,
    #     "layer_to_layer_buff": LARGE_BUFF,
    #     "output_neuron_color": WHITE,
    #     "input_neuron_color": WHITE,
    #     "hidden_layer_neuron_color": WHITE,
    #     "neuron_stroke_width": 2,
    #     "neuron_fill_color": GREEN,
    #     "edge_color": LIGHT_GREY,
    #     "edge_stroke_width": 2,
    #     "edge_propogation_color": YELLOW,
    #     "edge_propogation_time": 1,
    #     "max_shown_neurons": 16,
    #     "brace_for_large_layers": True,
    #     "average_shown_activation_of_large_layer": True,
    #     "include_output_labels": False,
    #     "arrow": False,
    #     "arrow_tip_size": 0.1,
    #     "left_size": 1,
    #     "neuron_fill_opacity": 1
    # }

    # Constructor with parameters of the neurons in a list
    def __init__(
        self,
        neural_network,
        *args,

            neuron_radius=0.20,
            neuron_to_neuron_buff=MED_SMALL_BUFF,
            layer_to_layer_buff=LARGE_BUFF,
            output_neuron_color=WHITE,
            input_neuron_color=WHITE,
            hidden_layer_neuron_color=WHITE,
            neuron_stroke_width=2,
            neuron_fill_color=GREEN,
            edge_color=LIGHT_GREY,
            edge_stroke_width=2,
            edge_propogation_color=YELLOW,
            edge_propogation_time=1,
            max_shown_neurons=16,
            brace_for_large_layers=True,
            average_shown_activation_of_large_layer=True,
            include_output_labels=False,
            arrow=False,
            arrow_tip_size=0.1,
            left_size=1,
            neuron_fill_opacity=1,

            **kwargs,
    ):
        VGroup.__init__(self, *args, **kwargs)

        self.neuron_radius = neuron_radius
        self.neuron_to_neuron_buff = neuron_to_neuron_buff
        self.layer_to_layer_buff = layer_to_layer_buff
        self.output_neuron_color = output_neuron_color
        self.input_neuron_color = input_neuron_color
        self.hidden_layer_neuron_color = hidden_layer_neuron_color
        self.neuron_stroke_width = neuron_stroke_width
        self.neuron_fill_color = neuron_fill_color
        self.edge_color = edge_color
        self.edge_stroke_width = edge_stroke_width
        self.edge_propogation_color = edge_propogation_color
        self.edge_propogation_time = edge_propogation_time
        self.max_shown_neurons = max_shown_neurons
        self.brace_for_large_layers = brace_for_large_layers
        self.average_shown_activation_of_large_layer = average_shown_activation_of_large_layer
        self.include_output_labels = include_output_labels
        self.arrow = arrow
        self.arrow_tip_size = arrow_tip_size,
        self.left_size = left_size
        self.neuron_fill_opacity = neuron_fill_opacity

        self.layer_sizes = neural_network
        self.add_neurons()
        self.add_edges()
        self.add_to_back(self.layers)

    # Helper method for constructor
    def add_neurons(self):
        layers = VGroup(*[
            self.get_layer(size, index)
            for index, size in enumerate(self.layer_sizes)
        ])
        layers.arrange_submobjects(RIGHT, buff=self.layer_to_layer_buff)
        self.layers = layers
        if self.include_output_labels:
            self.label_outputs_text()

    # Helper method for constructor

    def get_nn_fill_color(self, index):
        if index == -1 or index == len(self.layer_sizes) - 1:
            return self.output_neuron_color
        if index == 0:
            return self.input_neuron_color
        else:
            return self.hidden_layer_neuron_color
    # Helper method for constructor

    def get_layer(self, size, index=-1):
        layer = VGroup()
        n_neurons = size
        if n_neurons > self.max_shown_neurons:
            n_neurons = self.max_shown_neurons
        neurons = VGroup(*[
            Circle(
                radius=self.neuron_radius,
                stroke_color=self.get_nn_fill_color(index),
                stroke_width=self.neuron_stroke_width,
                fill_color=BLACK,
                fill_opacity=self.neuron_fill_opacity,
            )
            for x in range(n_neurons)
        ])
        neurons.arrange_submobjects(
            DOWN, buff=self.neuron_to_neuron_buff
        )
        for neuron in neurons:
            neuron.edges_in = VGroup()
            neuron.edges_out = VGroup()
        layer.neurons = neurons
        layer.add(neurons)

        if size > n_neurons:
            dots = Tex("\\vdots")
            dots.move_to(neurons)
            VGroup(*neurons[:len(neurons) // 2]).next_to(
                dots, UP, MED_SMALL_BUFF
            )
            VGroup(*neurons[len(neurons) // 2:]).next_to(
                dots, DOWN, MED_SMALL_BUFF
            )
            layer.dots = dots
            layer.add(dots)
            if self.brace_for_large_layers:
                brace = Brace(layer, LEFT)
                brace_label = brace.get_tex(str(size))
                layer.brace = brace
                layer.brace_label = brace_label
                layer.add(brace, brace_label)
        return layer

    """
    Highlights an entire layer of neurons
    index: The index of the layer. Input layer would be index 0
    """
    def highlight_layer(self, index = 0):
      print(self.layer_sizes)
      print(self.output_labels[0])
      if index == 0:
        return Indicate(self.output_labels[:self.layer_sizes[0]])
      else:
        num = self.layer_sizes[index]
        start = sum(self.layer_sizes[1:index])
        end = start + num
        return Indicate(self.output_labels[start:end])


    """
    Highlights all the edges between two layers of neurons
    index: Index 0 highlights edges between input layer and 1st hidden layer
    """
    def highlight_edges(self, index):
        return Indicate(self.edge_groups[index])

    # Helper method for constructor

    def add_edges(self):
        self.edge_groups = VGroup()
        for l1, l2 in zip(self.layers[:-1], self.layers[1:]):
            edge_group = VGroup()
            for n1, n2 in it.product(l1.neurons, l2.neurons):
                edge = self.get_edge(n1, n2)
                edge_group.add(edge)
                n1.edges_out.add(edge)
                n2.edges_in.add(edge)
            self.edge_groups.add(edge_group)
        self.add_to_back(self.edge_groups)
    # Helper method for constructor

    def get_edge(self, neuron1, neuron2):
        if self.arrow:
            return Arrow(
                neuron1.get_center(),
                neuron2.get_center(),
                buff=self.neuron_radius,
                stroke_color=self.edge_color,
                stroke_width=self.edge_stroke_width,
                tip_length=self.arrow_tip_size
            )
        return Line(
            neuron1.get_center(),
            neuron2.get_center(),
            buff=self.neuron_radius,
            stroke_color=self.edge_color,
            stroke_width=self.edge_stroke_width,
        )



    # Labels each input neuron with a char l or a LaTeX character
    def label_inputs(self, s):
        self.output_labels = VGroup()
        for n, neuron in enumerate(self.layers[0].neurons):
            l = s[n]
            label = Tex(f"{l}")
            label.set_height(0.3 * neuron.get_height())
            label.move_to(neuron)
            self.output_labels.add(label)
        self.add(self.output_labels)

    # Labels each output neuron with a char l or a LaTeX character
    def label_outputs(self, l):
        self.output_labels = VGroup()
        for n, neuron in enumerate(self.layers[-1].neurons):
            label = Tex(f"{l}_"+"{"+f"{n + 1}"+"}")
            label.set_height(0.4 * neuron.get_height())
            label.move_to(neuron)
            self.output_labels.add(label)
        self.add(self.output_labels)

    # Labels each neuron in the output layer with text according to an output list
    def label_outputs_text(self, outputs):
        self.output_labels = VGroup()
        for n, neuron in enumerate(self.layers[-1].neurons):
            label = Tex(outputs[n])
            label.set_height(0.75*neuron.get_height())
            label.move_to(neuron)
            label.shift((neuron.get_width() + label.get_width()/2)*RIGHT)
            self.output_labels.add(label)
        self.add(self.output_labels)

    # Labels the hidden layers with a char l or a LaTeX character
    def label_hidden_layers(self, s):
        self.output_labels = VGroup()
        for layer in self.layers[1:-1]:
            for n, neuron in enumerate(layer.neurons):
                l = s[n]
                label = Tex(f"{l}")
                label.set_height(0.3 * neuron.get_height())
                label.move_to(neuron)
                self.output_labels.add(label)
        self.add(self.output_labels)

In [ ]:
def ints_to_chars(l):
  out = []
  for i in l:
    out.append(str(i))
  return out

In [ ]:
input_layer_data = np.array([x[0][0], x[0][1]]).reshape(2,1)
print(input_layer_data)
# W1 = np.random.randn(2,2)
W1 = np.array([[]])
h = np.matmul(W1, input_layer_data)
h = np.round(h,1)
print(ints_to_chars(h.flatten()))
t = np.array([1.0,0.0,1.0,0.0,1.0,0.0]).reshape(6,1)

[[6. ]
 [0.9]]
['8.7', '0.2']


In [ ]:
# %%manim -ql -v WARNING MiniBatchSplit

# class MiniBatchSplit(MovingCameraScene):
#     def construct(self):
#         feature_colors = [RED, BLUE]
#         # the input X
#         X = Matrix(x).shift(LEFT*3)
#         X.set_column_colors(*feature_colors)

#         # 3 batches
#         batch_1_m = Matrix(x[:3])
#         batch_2_m = Matrix(x[3:6])
#         batch_3_m = Matrix(x[6:9])

#         buckets = VGroup(batch_1_m.shift(RIGHT*3, UP*2.5).scale(0.75), 
#                          batch_2_m.shift(RIGHT*3).scale(0.75), 
#                          batch_3_m.shift(RIGHT*3, DOWN*2.5).scale(0.75))

#         self.play(Create(X)) # puts main matrix on screen
#         self.wait()

#         self.play(Transform(X, buckets)) # shows splitting up of batches

#         self.wait()
#         self.play(FadeOut(X)) # removes main matrix from screen


#         batch_1_anno = Text("Batch 1").shift(UP*2)
#         self.play(FadeIn(batch_1_anno))

#         batch_1_m = Matrix(x[:3])
#         self.play(Create(batch_1_m))
#         self.play(batch_1_m.animate.shift(LEFT*5), batch_1_anno.animate.shift(LEFT*5))

#         input_layer_data = np.array([x[0][0], x[0][1]])
#         input_layer = Matrix(input_layer_data).shift(LEFT*5)
#         input_layer_anno = Text("Input Layer").shift(LEFT*5, UP*3)
#         self.play(Transform(batch_1_m, input_layer), Transform(batch_1_anno, input_layer_anno))


#         NN = NeuralNetworkMobject([6, 2, 1]).scale(2)
      
#         # NN.label_inputs(['6.0','0.0','1.0','0.6','2.0','0.3'])
#         NN.label_inputs(ints_to_chars(input_layer_data.flatten()))
#         self.play(Create(NN))
#         self.play(FadeOut(batch_1_m, batch_1_anno))
#         self.play(self.camera.frame.animate.shift(RIGHT*3))

#         NN.label_hidden_layers(ints_to_chars(h.flatten()))
#         self.play(NN.highlight_edges(0))
#         self.play(NN.highlight_layer(1))
#         self.wait()

In [ ]:
%%manim -ql -v WARNING MiniBatchSplit

class MiniBatchSplit(MovingCameraScene):
    def construct(self):
        feature_colors = [RED, BLUE]
        # the input X
        X = Matrix(x).shift(LEFT*3)
        X.set_column_colors(*feature_colors)

        # X_shuffled = Matrix(x_shuffled)

        # 3 batches
        batch_1_m = Matrix(x[:3])
        batch_2_m = Matrix(x[3:6])
        batch_3_m = Matrix(x[6:9])

        buckets = VGroup(batch_1_m.shift(RIGHT*3, UP*2.5).scale(0.75), 
                         batch_2_m.shift(RIGHT*3).scale(0.75), 
                         batch_3_m.shift(RIGHT*3, DOWN*2.5).scale(0.75))

        self.play(Create(X))
        self.wait()

        self.play(ReplacementTransform(X, buckets))

        self.wait()
        self.play(FadeOut(buckets))


        batch_1_anno = Text("Batch 1").shift(UP*2)
        self.play(FadeIn(batch_1_anno))

        batch_1_m = Matrix(x[:3])
        self.play(Create(batch_1_m))
        self.play(batch_1_m.animate.shift(LEFT*5), batch_1_anno.animate.shift(LEFT*5))

        # highlight x1
        self.play(Circumscribe(SurroundingRectangle(batch_1_m.get_rows()[0])))
        self.wait()
        batch_1_1_m = Matrix([[x[0][0]], [x[0][1]]]).shift(LEFT*2)
        # batch_1_1_anno = MathTex("x_1").shift(LEFT*2, UP)
        self.play(Create(batch_1_1_m))

        # highlight x2
        self.play(Circumscribe(SurroundingRectangle(batch_1_m.get_rows()[1])))
        self.wait()
        batch_1_2_m = Matrix([[x[1][0]], [x[1][1]]]).shift(LEFT*0.5)
        # batch_1_2_anno = MathTex("x_2").shift(LEFT*0.5, UP)
        self.play(Create(batch_1_2_m))

        # highlight x3
        self.play(Circumscribe(SurroundingRectangle(batch_1_m.get_rows()[2])))
        self.wait()
        batch_1_3_m = Matrix([[x[2][0]], [x[2][1]]]).shift(RIGHT)
        # batch_1_3_anno = MathTex("x_3").shift(RIGHT, UP)
        self.play(Create(batch_1_3_m))

        self.play(self.camera.frame.animate.shift(RIGHT*4))

        # move the 3 inputs to UL
        self.play(batch_1_1_m.animate.shift(UP*2.5), 
                  batch_1_2_m.animate.shift(UP*2.5),
                  batch_1_3_m.animate.shift(UP*2.5))


        self.wait()

        NN = NeuralNetworkMobject([2, 2, 1]).scale(2).shift(RIGHT*2) 
      
        # NN.label_inputs(['6.0','0.0','1.0','0.6','2.0','0.3'])
        NN.label_inputs(ints_to_chars(input_layer_data.flatten()))
        self.play(Create(NN))


        self.wait()
        NN.label_hidden_layers(ints_to_chars(h.flatten()))
        # highlight the first input
        self.play(Indicate(batch_1_1_m), NN.highlight_layer(0))

        
        self.play(NN.highlight_edges(0))
        self.play(NN.highlight_layer(1))


        mean_formula =MathTex(r"\mu = \frac{1}{n}\sum_i{z_i}").shift(UP*3,RIGHT*5)
        self.play(Write(mean_formula))
        mean_formula_expanded =MathTex(r"\mu = \frac{1}{2}(8.7+0.2)").shift(UP*3,RIGHT*5)
        self.play(Transform(mean_formula, mean_formula_expanded))
        # h1_equation = MathTex(r"h_1 = w^{(1)}_{1,1} \cdot x_{1,1} + w^{(1)}_{2,1} \cdot x_{1,2} + b_{1, 1}").shift(UP*3, RIGHT*3)
        # h2_equation = MathTex(r"h_2 = w^{(1)}_{1,2} \cdot x_{1,1} + w^{(1)}_{2,2} \cdot x_{1,2} + b_{1, 2}").shift(UP*2, RIGHT*3)

        # self.play(Write(h1_equation), Write(h2_equation))

        # hidden_layer_table = Table([['1', '2'], ['3', '4'], ['5', '6']], col_labels=[MathTex("h_1"), MathTex("h_2")]).shift(RIGHT*8.5, UP*2).scale(0.75)
        # self.play(Create(hidden_layer_table))
        # self.play(Indicate(batch_1_1_m), Indicate(hidden_layer_table.get_rows()[1])) # TODO INDICATE THE EDGES
        # self.play(Indicate(batch_1_2_m), Indicate(hidden_layer_table.get_rows()[2])) # TODO INDICATE THE EDGES
        # self.play(Indicate(batch_1_3_m), Indicate(hidden_layer_table.get_rows()[3])) # TODO INDICATE THE EDGES



        self.wait()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:222: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:222: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:255: DeprecationWarning: This method is not guaranteed to stay around. Please prefer getting the attribute normally.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:255: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().


[2, 2, 1]
Tex('8.7')


[2, 2, 1]
Tex('8.7')
